In [3]:
import numpy as np
import pandas as pd
import nltk
import json, gzip
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
true = True
false = False
from tqdm import tqdm
import random

from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamulticore import LdaMulticor

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhengzeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zhengzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zhengzeng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [81]:
#read in data
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

data = parse("Home_and_Kitchen.json.gz")

        
def sub_sample_data(data,data_size):
    reviews = {}
    index = 0
    for d in data:
        if index %data_size ==0:
            entry = dict(d)
            reviews[index] = entry.get('reviewText')
        index = index+1
    return reviews
#     print(dic)
reviews = sub_sample_data(data,10000)
print("finish loading review data ", len(reviews))
review_data = pd.DataFrame.from_dict(reviews,orient = "index")
print(review_data.head(20))




finish loading review data  2193
                                                        0
0       I don't use these for their original use, and ...
10000   Safety valve leaked water / steam. Returned th...
20000   I love this vacuum!  When I first got it, I re...
30000   Got these for my husband as a belated Father's...
40000   This really has made pouring pancakes much eas...
50000   These stainless steel measuring spoons beat pl...
60000   This pot is great. My wife was getting on to m...
70000   Amazon's product review says these are 7 x 10 ...
80000   The gradated shapes are exactly what I needed ...
90000   This canner is great!  It works well.  I reall...
100000  We juice oranges from 2 trees every year, and ...
110000  I bought these basically because I have a smal...
120000  I've owned about a dozen pepper mills over the...
130000                                         works good
140000  I'd prefer to have a newer model microwave, bu...
150000  A great thermometer. If only th

In [82]:
review_data.columns = ['reviewText']
review_data['text'] = ' '.join(review_data['reviewText'].astype(str))
corpus = review_data['text'].values
# print(corpus.astype(str))

len(corpus)


2193

In [8]:
review_data.head(10)

,reviewText,text
0,"I don't use these for their original use, and ...","I don't use these for their original use, and ..."
1,"Seems a bit expensive for a plastic bottle, bu...","I don't use these for their original use, and ..."
2,"Great product, love it!!","I don't use these for their original use, and ..."
3,This is a sturdy floating corner shelf! We mo...,"I don't use these for their original use, and ..."
4,I purchased 4 of these shelves. they look grea...,"I don't use these for their original use, and ..."
5,I truly wanted to like this item. It fell apa...,"I don't use these for their original use, and ..."
6,"Pretty flimsy, but does the job. If your corne...","I don't use these for their original use, and ..."
7,"Note, the way this shelf mounts to the wall is...","I don't use these for their original use, and ..."
8,It's a shelf. it was inexpensive and holds a ...,"I don't use these for their original use, and ..."
9,Instructions are wrong and it is not very sturdy.,"I don't use these for their original use, and ..."


In [88]:

# The WordNet Lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

# Acceptable parts of speech
pos_map = {'N': wordnet.NOUN, 'J': wordnet.ADJ, 'V': wordnet.VERB, 'R': wordnet.ADV }

# produce a bag of words using lemmatization and tokenization
def clean_corpus(list_of_text):
    cleaned = []
    for i in tqdm(range(300)):
        words = []
        for sent in nltk.sent_tokenize(list_of_text[i+7]): #sentences in a review 
            for (word, pos) in nltk.pos_tag(nltk.word_tokenize(sent)):
                if not pos[0] in pos_map:
                    continue
                word = lemmatizer.lemmatize(word.lower(), pos_map[pos[0]])
                words.append(word.lower())
        cleaned.append(words)
    return cleaned

# This process will take some time to complete
clean_texts = clean_corpus(corpus)


100%|██████████| 300/300 [27:30<00:00,  5.50s/it]


In [90]:
#check the first corpus
clean_texts[0]

In [91]:
from collections import Counter

# Print the 30 most frequent words, and add to stopword list
counter = Counter([w for t in clean_texts for w in t])
wordcounts = counter.most_common(n=30)
stopwords = []
for word, count in wordcounts:
    stopwords.append(word)
    print('%s%i' % (word.ljust(15), count))
    
# Add some new stopworks to stopword list
stopwords.extend(['this', 'it', 'the', 'that','do','n\'t','item','then','no','not','up','down','on','in','get','even','find','soon','le'])

be             948600
have           338400
do             219600
not            186900
use            178200
very           159900
n't            158400
great          152100
love           134100
get            117600
so             115500
good           111900
work           109500
make           108600
's             102900
just           99900
well           93600
buy            92700
look           91200
time           82200
up             78000
easy           76200
nice           74400
product        71100
more           64800
out            57900
really         57900
quality        57000
much           55500
need           53100


In [92]:
# Remove stopwords from each lemmatized text block
for i in range(len(clean_texts)):
    j = 0
    while j < len(clean_texts[i]):
        if clean_texts[i][j] in stopwords:
            del clean_texts[i][j]
        else:
            j += 1

In [94]:
# print(clean_texts[0])

In [95]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
dictionary = Dictionary(clean_texts)
corpus = [dictionary.doc2bow(text) for text in clean_texts]


In [96]:
from gensim.models.ldamulticore import LdaMulticore

# Pretrain the model using the corpus and number of topics
#measure the coherence

num_topics = range(10, 30, 2)
# Alpha parameter

def build_lda_model(corpus, dictionary,text, num_topic):
    lda = LdaMulticore(corpus, id2word=dictionary, num_topics=num_topic, passes=2)
    cm = CoherenceModel(lda, corpus=corpus, dictionary=dictionary, texts =text,coherence='c_v')
    coherence = cm.get_coherence()
    print("for model with %d topics: coherence is %f" %(num_topic,coherence))
    return lda

In [ ]:
#re run the new data into the model, so the coherence result may not the same as in the txt file. But they show the same trend.
models = []
for num in num_topics:
    model = build_lda_model(corpus,dictionary,clean_texts,num)

for model with 10 topics: coherence is 0.238590
for model with 12 topics: coherence is 0.238076
for model with 14 topics: coherence is 0.233729
for model with 16 topics: coherence is 0.240597


In [79]:
ldamodel = build_lda_model(corpus,dictionary,clean_texts,28)

for model with 28 topics: coherence is 0.372404


In [77]:
for i in range(10):
    print('Topic %i: %s\n' % (i, ldamodel.print_topic(i, topn=10)))

Topic 0: 0.016*"find" + 0.014*"issue" + 0.014*"hard" + 0.012*"place" + 0.011*"nice" + 0.011*"really" + 0.010*"long" + 0.010*"anything" + 0.010*"take" + 0.009*"size"

Topic 1: 0.019*"make" + 0.017*"cover" + 0.013*"easy" + 0.013*"tea" + 0.013*"lid" + 0.012*"best" + 0.011*"take" + 0.010*"hard" + 0.010*"page" + 0.009*"new"

Topic 2: 0.017*"coffee" + 0.016*"nice" + 0.009*"keep" + 0.009*"hot" + 0.008*"new" + 0.008*"size" + 0.008*"reminder" + 0.007*"much" + 0.007*"too" + 0.007*"price"

Topic 3: 0.014*"'m" + 0.014*"daughter" + 0.012*"little" + 0.012*"page" + 0.011*"quality" + 0.011*"come" + 0.011*"expect" + 0.009*"make" + 0.009*"uderzo" + 0.009*"get"

Topic 4: 0.016*"make" + 0.015*"more" + 0.013*"keep" + 0.012*"nativity" + 0.012*"order" + 0.011*"http" + 0.011*"poster" + 0.011*"small" + 0.011*"go" + 0.008*"friend"

Topic 5: 0.017*"cup" + 0.014*"star" + 0.013*"out" + 0.010*"family" + 0.010*"want" + 0.010*"come" + 0.010*"one" + 0.010*"more" + 0.009*"next" + 0.009*"get"

Topic 6: 0.013*"series" + 

In [80]:
#saving the model
import joblib
joblib.dump(ldamodel,'./topic.model.pkl')

['./topic.model.pkl']